In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
# [https://github.com/metmuseum/openaccess/raw/master/MetObjects.csv]
# (https://github.com/metmuseum/openaccess/raw/master/MetObjects.csv)


In [ ]:
path = "Sales_US.csv"
df = pd.read_csv(path,parse_dates = ['Order Date', 'Ship Date'])
df.head(3)

In [ ]:
df.columns

In [ ]:
def clean_currency(x):
    if isinstance(x, str):
        x = x.strip().replace('$', '').replace(',', '')
        if x.startswith('('):
            x = '-'+ x.strip('()')
    return x

In [ ]:
for i in ['Profit', 'Sales', 'Sales Forecast']:
    df[i] = df[i].apply(clean_currency).astype('float')

#### Most profitable sub-category in last 12 month sales

In [ ]:
(df[pd.DatetimeIndex(df['Ship Date']).year == 2021][['Sub-Category', 'Profit']]
   .groupby('Sub-Category').agg({'Profit': np.sum})
   .sort_values(by = 'Profit',ascending = False)).index[0]

#### Visualization of sales per month v/s same month last year sales

In [ ]:
twoyrssales_df = df[pd.DatetimeIndex(df['Ship Date']).year >= 2020][['Ship Date', 'State', 'Sales', 'Profit']]
twoyrssales_df['Year'] = pd.DatetimeIndex(twoyrssales_df['Ship Date']).year
twoyrssales_df['Month'] = pd.DatetimeIndex(twoyrssales_df['Ship Date']).month
twoyrssales_df = twoyrssales_df.loc[twoyrssales_df['Year'] < 2022]

In [ ]:
twoyrssales_df = twoyrssales_df[['Year', 'Month', 'State', 'Sales']]
twoyrssummary_df = twoyrssales_df.groupby(['Year', 'Month','State']).sum()

In [ ]:
twoyrssummary_df

In [ ]:
twoyrssummary_df = twoyrssummary_df.unstack(level = -3, fill_value = 0)
twoyrssummary_df

In [ ]:
twoyrssummary_df = twoyrssummary_df.Sales.reset_index()
twoyrssummary_df

In [ ]:
def subplot_by_colval(indf,subplotcol,xcol,ycol,**kwargs):
    colval = indf[subplotcol].sort_values().unique()
    for c in colval:
        querystr = subplotcol+' == '+str(c)
        chart_title = subplotcol+': '+str(c)
        indf.query(querystr).plot(xcol,ycol,title = chart_title, **kwargs)
subplot_by_colval(twoyrssummary_df,'Month','State',[2020,2021],kind = 'bar', figsize = (16,5), fontsize = 10, rot = 65)

#### Churn rate per state
#### if customer doesn't purchase any thing from stores within 180 days form his/her last purchase, then we consider as a 
#### churned customer.

In [ ]:
churn_df = df.groupby(['State','Customer ID'])['Ship Date'].max().reset_index()

In [ ]:
churn_df['days_from_lastpurchase'] = df['Ship Date'].max() - churn_df['Ship Date']

In [ ]:
churn_df['days_from_lastpurchase'] = churn_df['days_from_lastpurchase'].apply( lambda x: x.days) #convert timedelta to int

In [ ]:
churn_period = 180
churn_df.loc[churn_df['days_from_lastpurchase'] > churn_period, 'Category'] = 'Churned'
churn_df.loc[churn_df['days_from_lastpurchase'] < churn_period, 'Category'] = 'Survived'

In [ ]:
churn_df.head(30)

In [ ]:
churn_rate_df = (churn_df[['State','Category', 'days_from_lastpurchase']].groupby(['State','Category']).agg(['count'])
            .reset_index())
churn_rate_df

In [ ]:
churn_rate_df = churn_rate_df.pivot(columns='Category',values = ['days_from_lastpurchase'],index='State').fillna(0)
churn_rate_df

In [ ]:
churn_rate_df = churn_rate_df['days_from_lastpurchase'].reset_index()

In [ ]:
churn_rate_df['churn_rate'] = (round((churn_rate_df['Churned'] *100) / (churn_rate_df['Churned'] + 
                                                                        churn_rate_df['Survived']),0))
churn_rate_df.sort_values(by='churn_rate', ascending=True)

In [ ]:
churn_rate_df[['Churned','Survived']].sum()

#### Boxplot of churned customers per state

In [ ]:
#churn_df = churn_df[['State', 'days_from_lastpurchase','Category']]
churn_pos_df = churn_df.loc[churn_df['Category'] == 'Churned']
sns.set(rc={'figure.figsize':(22, 5)})
x = sns.boxplot(data = churn_pos_df, x ='State',y ='days_from_lastpurchase')
x.set_xticklabels(labels = churn_pos_df.State.unique(), rotation = 75,fontsize = 16);

#### Boxplot Visualization of Survived customers

In [ ]:
churn_neg_df = churn_df.loc[churn_df['Category'] == 'Survived']
sns.set(rc={'figure.figsize':(22, 5)})
x = sns.boxplot(data = churn_neg_df, x ='State',y ='days_from_lastpurchase')
x.set_xticklabels(labels = churn_neg_df.State.unique(), rotation = 75,fontsize = 16);

#### Customer Lifetime Value (CLV)

#### By Sales

In [ ]:
(df.groupby(['Customer ID','Customer Name']).agg({'Sales':np.sum, 'Profit':np.sum}).
 sort_values(by = 'Sales',ascending = False))

#### By Profit

In [ ]:
(df.groupby(['Customer ID','Customer Name']).agg({'Sales':np.sum, 'Profit':np.sum}).
 sort_values(by = 'Profit',ascending = False))

## Most profitable category and sub-category by segments

In [ ]:
(df.groupby(['Category', 'Sub-Category','Segment' ]).agg({'Profit': np.sum}).sort_values(by='Profit',
                                                                                                           ascending=False))

#### Visualization of sold products per month v/s same month last year sold products

In [ ]:
soldproducts_df = (df.loc[(pd.DatetimeIndex(df['Ship Date']).year < 2022) & (pd.DatetimeIndex(df['Ship Date']).year > 2019)]
 [['State','Product ID','Quantity', 'Ship Date']])

In [ ]:
#soldproducts_df['Year'] = pd.DatetimeIndex(soldproducts_df['Ship Date']).year
soldproducts_df['Year'] = soldproducts_df['Ship Date'].apply( lambda x : x.strftime('%Y'))
soldproducts_df['Month'] = pd.DatetimeIndex(soldproducts_df['Ship Date']).month

In [ ]:
soldproducts_df

In [ ]:
soldproducts_df = soldproducts_df.groupby(['Year', 'Month', 'State']).agg({'Quantity': np.sum})

In [ ]:
soldproducts_df = soldproducts_df.unstack(level = -3).fillna(0).reset_index()

In [ ]:
soldproducts_df.columns = ['_'.join(c) for c in soldproducts_df.columns.values]

In [ ]:
(subplot_by_colval(soldproducts_df,'Month_','State_',['Quantity_2020','Quantity_2021'],kind = 'bar', figsize = (16,5),
                   fontsize = 10, rot = 65))